# Priprava podatkov

In [6]:
import numpy as np
import pandas as pd

Preberemo datoteko

In [7]:
xls = pd.ExcelFile(r'IJS_database_anonimized.xlsx')
sheet1 = xls.parse(0)
sheet2 = xls.parse(1)
sheet3 = xls.parse(2)

frames = [sheet1, sheet2, sheet3]
df = pd.concat(frames)

Izračunamo starost

In [8]:
from datetime import datetime
#tu bomo uredili stolpce relevantne za starost pri posegu in slednje izračunali

df.rename(columns = {'Datum rojstva':'Datum_rojstva'}, inplace = True)  
df.rename(columns = {'Datum posega':'Datum_posega'}, inplace = True) 
df.at[0,'Datum_rojstva'] = "29.4.1948"
df['Datum_rojstva'] = df['Datum_rojstva'].str.replace(" ", "")
df['Datum_posega'] = df['Datum_posega'].str.replace(" ", "")



def calculate_age(datum_rojstva, datum_posega):
    datum_rojstva = datetime.strptime(datum_rojstva, "%d.%m.%Y").date()
    datum_posega = datetime.strptime(datum_posega, "%d.%m.%Y").date()
    return datum_posega.year - datum_rojstva.year - ((datum_posega.month, datum_posega.day) < (datum_rojstva.month, datum_rojstva.day))


df['starost'] = df.apply(lambda x: calculate_age(x.Datum_rojstva, x.Datum_posega), axis=1)

df.drop('Datum_posega', axis=1, inplace=True)
df.drop('Datum_rojstva', axis=1, inplace=True)

Dodamo cifro excela

In [9]:
#df.columns = ['cifra_pacienta',"spol","glavna_diagnoza","responder","ef","lvedd","vti","Na","K","secnina","kreatinin","ast","AF","gGT","bilirubin_cel","pro_BNP","CRP","razdalja","holesterol","HDL","trigliceridi","glukoza","starost"]
transplantation_method = pd.Series([])
df = df.reset_index()

for i in range(len(df)):
    if df["Unnamed: 0"][i] in range(1,30):
        transplantation_method[i]=0
    else:
        transplantation_method[i]=1

df.insert(2, "transplantation_method", transplantation_method)

C:\Users\borbr\AppData\Local\Temp\ipykernel_4564\758837559.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  transplantation_method = pd.Series([])


Stolpce, kjer manjka več kot petina podatkov izbrišemo

In [10]:
manjkajoce = df.isnull().sum()

meja = 0.3
counter = 0 #steje koliko stolpcev smo pometali ven
for col in df.columns:
    if manjkajoce[col] > 0:
        if manjkajoce[col] < len(df)*meja:
            pass
        else:
            print(col, round(manjkajoce[col]/len(df[col]),2))
            df.drop(col, axis=1, inplace=True)
            counter += 1
    
#print(df.isnull().sum())

ALT 0.78
bilirubin (dir) 0.88
Način TX 0.76
urat 0.47
Fe 0.47
EDV 0.35
ESV 0.37
feritin 0.42


In [11]:
df.drop(df.filter(regex="Unname"),axis=1, inplace=True)
df.drop(df.filter(regex="cifra_pacienta"),axis=1, inplace=True) # znebimo se še unnamed index stolpca
df.drop(df.filter(regex="index"),axis=1, inplace=True) # indeks pride od drugacnih listov (vsak list posebej zacne stevilciti) in ni pomemben
df.drop(df.filter(regex="Komentar kontrole"),axis=1, inplace=True)#ta ni prav lep za ML

Kategoricne damo v 0 1

In [12]:
df_one = pd.get_dummies(df["Spol"])
df_two = pd.concat((df_one, df), axis=1)
df_two = df_two.drop(["Spol"], axis=1)
df_two = df_two.drop(["M"], axis=1)
df = df_two.rename(columns={"Ž": "Spol"})

df_one = pd.get_dummies(df["Glavna diagnoza"])
df_two = pd.concat((df_one, df), axis=1)
df_two = df_two.drop(["Glavna diagnoza"], axis=1)
df_two = df_two.drop(["Neishemična KMP"], axis=1)
df = df_two.rename(columns={"Ishemična KMP": "Glavna_diagnoza"})

Koliko vrstic smo pometali ven (ce manjka vec kot meja.vrstice podatkov)

In [13]:
meja_vrstice = 0.3 #manjka vec kot 70% v vrstici
vse_vrstice = df.shape[0]
df = df[df.isnull().sum(axis=1) < meja_vrstice * len(df.columns)]
nove_vrstice = df.shape[0]
print("Pometane vrstice (v procentih)", round(((vse_vrstice - nove_vrstice) / vse_vrstice),2))

#Koliko stolpcev smo pometali ven
print("Pometani stolpci (v procentih)", round(((counter + 1) / (len(df.columns)+counter)),2))#pri counter +1 ce ohranimo stolpec komentar, ki je tezko uporaben

Pometane vrstice (v procentih) 0.18
Pometani stolpci (v procentih) 0.24


Spremenimo se imena stolpcev, ker so tako uporabljeni v nadaljni kodi

In [14]:
#df.columns = ['cifra_pacienta',"spol","glavna_diagnoza","responder","ef","lvedd","vti","Na","K","secnina","kreatinin","ast","AF","gGT","bilirubin_cel","pro_BNP","CRP","razdalja","holesterol","HDL","trigliceridi","glukoza","starost"]
df=df.rename(columns = {'Responder':'responder'})

In [15]:
df.to_excel("data.xlsx")